In [1]:
""" This notebook computes optimal portfolio weights for NYSE_1 dataset in the case of logarithmic utility.
The results for 30 experiments of the GDSEG algorithm are written to the file log_opt_portf_NYSE_1.txt
"""
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
# Importing NYSE_1 dataset
stocks=pd.read_csv('NYSE_1.csv')
print(stocks.shape)
stocks.head()

(5651, 36)


,ahp,alcoa,amerb,arco,coke,comme,dow,dupont,espey,exxon,...,merck,mmm,mobil,morris,pandg,pills,schlum,sears,sherw,tex
0,1.01515,1.02765,1.04183,1.02083,1.00637,1.04938,1.00847,1.01983,1.05426,0.99751,...,1.03148,1.03377,1.01018,1.01495,1.00775,1.00526,1.01176,1.00578,0.99697,0.99752
1,1.01493,1.04036,0.98905,0.99490,1.00475,0.95294,1.00840,1.00833,1.04412,1.00500,...,1.00898,1.00251,1.01259,1.00000,1.00192,1.00000,1.01938,1.00958,0.99088,1.00248
2,1.00000,0.97629,0.97786,0.99744,0.98583,0.98765,0.99722,0.99449,0.97183,1.00000,...,0.98043,0.95990,0.99751,0.97218,0.98656,0.98429,0.97338,1.00000,1.02761,0.99752
3,1.02451,1.00662,1.02642,1.00257,1.01917,1.00000,0.99443,1.00693,1.00000,1.00000,...,1.01089,1.03655,1.01247,0.99663,1.00778,1.01596,1.00000,1.00190,1.00299,1.02233
4,1.03100,0.98465,1.00368,1.00513,1.00313,1.05000,1.02801,1.00413,1.05797,1.01741,...,1.01077,0.99496,0.99507,0.98649,1.01158,0.99738,1.01563,1.01515,1.01190,1.00971


In [3]:
# r: array for stock returns
N=stocks.shape[0]
d=stocks.shape[1]
r=np.zeros((N,d))
r=stocks.to_numpy()

In [4]:
def GDSEG(U,R,n_attempts=10**4,threshold=1/10**10):
    """ Greedy doubly stochastic exponentiated graient algorithm (GDSEG)
    U: empirical utility, power or logarithmic 
    R: array of stock pirces
    The function returns the optimal oprtfolio weights: w_old, the optimal value of the objective function: U(w_old,R), and the number of iterations: i
    """
    N=R.shape[0]
    d=R.shape[1]
    w_old=np.ones(d)/d
    w_new=np.zeros(d)
    attempt=0
    i=0
    while attempt<=n_attempts:
        i+=1
        k=np.random.randint(0,N)
        eta=np.random.rand()
        a=[w_old[j]*np.exp(eta*R[k,j]/(np.dot(w_old,R[k,:]))**(1-alpha)) for j in range(d)]
        w_new=a/np.sum(a)
        attempt+=1
        if U(w_new,R)>U(w_old,R)+threshold:
            w_old=w_new
            attempt=0
    return w_old, U(w_old,R), i

In [6]:
def U(nu,r):
    """ Empirical utility, power (0<alpha<=1) or logarithmic (alpha=0) """
    if alpha==0:
        return np.mean(np.log(np.dot(r[0:N,:],nu)))
    else:
        return np.mean(np.dot(r[0:N,:],nu)**alpha)

In [16]:
# Computation of optimal portfolios
n_experiments=30
alpha=0.0
opt_portf=np.zeros((n_experiments,d))
opt_val=np.zeros(n_experiments)
n_iter=np.zeros(n_experiments)
f=open('log_opt_portf_NYSE_1.txt','ab')
for s in range(n_experiments):
    np.random.seed(1+s)
    opt_portf[s,:], opt_val[s], n_iter[s]  = GDSEG(U,r)
    print('GDSEG experiment:',s,'Number of iterations:',n_iter[s])
    np.savetxt(f,opt_portf[s,:].reshape(1,d))
f.close()

GDSEG experiment: 0 Number of iterations: 231636.0
GDSEG experiment: 1 Number of iterations: 338989.0
GDSEG experiment: 2 Number of iterations: 341570.0
GDSEG experiment: 3 Number of iterations: 340095.0
GDSEG experiment: 4 Number of iterations: 320834.0
GDSEG experiment: 5 Number of iterations: 292283.0
GDSEG experiment: 6 Number of iterations: 350065.0
GDSEG experiment: 7 Number of iterations: 353001.0
GDSEG experiment: 8 Number of iterations: 319599.0
GDSEG experiment: 9 Number of iterations: 206623.0
GDSEG experiment: 10 Number of iterations: 303142.0
GDSEG experiment: 11 Number of iterations: 288378.0
GDSEG experiment: 12 Number of iterations: 255459.0
GDSEG experiment: 13 Number of iterations: 360453.0
GDSEG experiment: 14 Number of iterations: 285077.0
GDSEG experiment: 15 Number of iterations: 304645.0
GDSEG experiment: 16 Number of iterations: 241903.0
GDSEG experiment: 17 Number of iterations: 245023.0
GDSEG experiment: 18 Number of iterations: 306372.0
GDSEG experiment: 19 N

In [96]:
print('Average number of iterations',n_iter.mean())

Average number of iterations 282748.56666666665
